In [212]:
import pandas as pd
import numpy as np

In [213]:
df_pbp = pd.read_csv(r"RAW/nhl_pbp_20212022.csv")
df_shifts = pd.read_csv(r"RAW/nhl_shifts_20212022.csv")

# Cleaning play-by-play data

The shift data is already quite clean but the play-by-play data needs a lot of work before we are ready to start aggregating it by player

In [214]:
df_pbp.head(5)

,Unnamed: 0,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,0,20001,2021-10-12,1,PSTR,Period Start- Local time: 7:43 EDT,0:00,0.0,5x5,NaN,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,NaN,NaN,Jon Cooper,Mike Sullivan
1,1,20001,2021-10-12,1,FAC,PIT won Neu. Zone - PIT #77 CARTER vs TBL #21 ...,0:00,0.0,5x5,Neu,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,0.0,0.0,Jon Cooper,Mike Sullivan
2,2,20001,2021-10-12,1,HIT,"TBL #18 PALAT HIT PIT #77 CARTER, Off. Zone",0:18,18.0,5x5,Off,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,46.0,40.0,Jon Cooper,Mike Sullivan
3,3,20001,2021-10-12,1,STOP,PUCK IN NETTING,0:38,38.0,5x5,NaN,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,NaN,NaN,Jon Cooper,Mike Sullivan
4,4,20001,2021-10-12,1,FAC,TBL won Neu. Zone - PIT #9 RODRIGUES vs TBL #7...,0:38,38.0,5x5,Neu,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,0.0,0.0,Jon Cooper,Mike Sullivan


In [215]:
#Drop all unnecessary columns
columns = ['Game_Id','Date','Period','Event', 'Description', 'Strength', 'Type',
      'Away_Team', 'Home_Team', 'p1_name', 'p1_ID',
       'p2_name', 'p2_ID', 'p3_name', 'p3_ID']

df_pbp = df_pbp[columns]

In [216]:
# We have three goals that are missing a 'type' - I could just drop these but since the number is so low,
# I decided to manually fill it in by watching the game highlights to see how each goal was scored.
list_shots = ['SNAP SHOT', 'BACKHAND', 'BACKHAND']
df_pbp.iloc[[96417, 110893, 225472],6] = list_shots

In [217]:
#Include the type of shot used in the shot on goal or goal

df_pbp['Event'].replace('MISS', 'SHOT', inplace = True) # I decided to group in shots-on-goal with misses

def concat_event_type(row):
    #shot or goal type
    if row['Event'] in ['SHOT', 'GOAL']:
        return f"{row['Event']} ({row['Type']})"
    #fighting or regular penalty
    elif row['Event'] == 'PENL' and row['Type'] in ['Fighting (maj)', 'Instigator(2 min)',
                                                'Instigator - Misconduct(10 min)', 'Aggressor(10 min)'
                                               ]:
        return f"{row['Event']} (FIGHTING)"
   
    elif row['Event'] == 'PENL':
        return f"{row['Event']} (OTHER)"
    else:
        return row['Event']

# apply the function to create a new column
df_pbp['Event_New'] = df_pbp.apply(concat_event_type, axis=1)

#drop original Event and Type Column as they are no longer needed
df_pbp.drop(['Event', 'Type'], axis='columns', inplace=True)


In [218]:
#Let's now filter only for the events that we care about (goals, hit, shots, takeaways, giveaways, etc.)
relevant_events = ['HIT','SHOT (WRIST SHOT)', 'TAKE', 'BLOCK',
       'SHOT (SLAP SHOT)', 'SHOT (BACKHAND)', 'SHOT (TIP-IN)',
       'SHOT (SNAP SHOT)', 'GIVE','GOAL (SNAP SHOT)',
       'GOAL (WRIST SHOT)', 'SHOT (WRAP-AROUND)', 'SHOT (DEFLECTED)',
       'PENL (OTHER)', 'GOAL (TIP-IN)', 'GOAL (BACKHAND)', 'GOAL (DEFLECTED)', 'GOAL (SLAP SHOT)',
       'PENL (FIGHTING)', 'GOAL (WRAP-AROUND)']

df_pbp = df_pbp.loc[df_pbp['Event_New'].isin(relevant_events)]


# Creating Aggregate Dataframe

In [219]:
#Aggregating assists by player
def total_assists_by_player():
    df_filtered = df_pbp.loc[(df_pbp['Event_New'].str.startswith('GOAL')) & (df_pbp['p2_ID'].notnull() | df_pbp['p3_ID'].notnull())]
    assists = pd.concat([df_filtered['p2_ID'], df_filtered['p3_ID']])
    player_total_assists = assists.value_counts()
    df_assists= pd.DataFrame({'Player_Id': player_total_assists.index, 'Assists': player_total_assists.values})
    aggregate = pd.merge(df_shifts.groupby('Player_Id').first()['Player'].reset_index(), 
                         df_assists, on = 'Player_Id', how = 'left')
    return aggregate

aggregate = total_assists_by_player()

In [224]:
#Calculating total number of shifts by player
shifts = df_shifts.groupby('Player_Id')['Player_Id'].count()
player_total_shifts = pd.DataFrame({'Player_Id': shifts.index, 'Shifts': shifts.values})
aggregate2 = pd.merge(aggregate, player_total_shifts, on ='Player_Id', how = 'left')


In [ ]:
def create_event_count_df(df, event_col, p1_ID, p2_ID):
    # Create a new dataframe with index as the union of ids in p1_ID and p2_ID
    id_union = set(df[p1_ID]).union(set(df[p2_ID]))
    event_counts = pd.DataFrame(index=id_union)

    # Group the dataframe by event and player, then count the occurrences of each id
    counts = df.groupby([event_col, 'player'])[p1_ID].count().reset_index()
    counts.columns = [event_col, 'player', 'count']

    # Pivot the counts dataframe to create a new dataframe with one column for each event
    event_pivot = counts.pivot_table(index='player', columns=event_col, values='count', fill_value=0)

    # Merge the event count into the main dataframe
    event_counts = event_counts.merge(event_pivot, left_index=True, right_index=True, how='outer')

    return event_counts

event_counts = create_event_count_df(df_pbp, 'Event_New', 'p1_ID', 'p2_ID')

